# analytical and numerical approaches to ODEs

Analyzing an ODE (or a set of coupled ODEs) is in some ways slightly easier than what we have done for discrete-time models. The rules for computing equilibria and analyzing stability are different; you must keep them straight in your mind. Finding good, efficient numerical solutions is much harder, but luckily the tools you need are already available in Python, so it's not much harder in practice.


## analytical approaches

In general the steps for analyzing an ODE/set of ODEs are as follows:

- make sure you understand the meanings of the state variables and parameters, and what their units are;
- solve for equilibria, i.e. solve $dx/dt=0$, or for a set of equations solve the *simultaneous* equations $\{dx/dt=0,dy/dt=0,\ldots\}$. There is often an equilibrium at zero; sometime other equilibria are hard to solve for in closed form.
- evaluate the stability of equilibria
     - graphically
     - numerically, by evaluating the Jacobian. 
          - univariate models: $J = d(dx/dt)/dx$
          - multivariate models: if $\mathbf x$ is the state vector and $\mathbf g$ is the gradient vector, $\mathbf J = ( \partial g_i/\partial g_j )$ (analogous to the discrete-time Jacobian)
     - the equilibrium is stable if $J<0$ or the real part of the dominant eigenvalue of $\mathbf J$ is $<0$, unstable if $>0$.
- try to find time-dependent solutions by solving (integrating) the differential equation

## Allee effects

[Allee effects](https://en.wikipedia.org/wiki/Allee_effect) describe the phenomena where organisms do poorly (have lower *per capita* birth rates/higher *per capita* death rates) at low densities than at intermediate densities (in general, they will also do worse when population densities are very large). A reasonable mathematical model for an Allee effect  is

$$
\frac{dN}{dt} = r N \left( \frac{N}{A} - 1 \right) \left(1- \frac{N}{K} \right), \qquad r>0, K>0, 0<A<K
$$

- What are the units of the parameters $r$, $K$, $A$?
- What are the equilibria? 
- Draw a graph of $dN/dt$ vs. $N$ and make conclusions about the stability of the equilibria.
- Derive the Jacobian and evaluate it at the equilibria to confirm your conclusions from the previous step.

This can be separated and might be solvable by partial fractions and brute force:

$$
\int \frac{dN}{N \left( \frac{N}{A} - 1 \right) \left(1- \frac{N}{K} \right)} = \int r \, dt
$$
but we're not going to try that now.

## numerical solutions

First we need to define a *gradient function*:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
def gradfun(N, t, params):
    """cubic Allee effect model
    arguments *must* be in order (state vector, time, parameters)
    """
    r, K, A = params
    return(r*N*(N/A-1)*(1-N/K))


Define some parameters and a range of $N$ values (not $t$ values!) to plot the gradient function: because we are using numpy arrays, the computation is automatically done for all values of $N$.

In [ ]:
params = (1,3,2)
Nvec = np.arange(0,3.5,0.1)
gradvec = gradfun(Nvec, 0, params)

In [ ]:
%matplotlib inline
plt.plot(Nvec,gradvec)
plt.axhline(0,c="black",linestyle="--");

Now use the `scipy.integrate.odeint` function to integrate the differential equation numerically ...

In [ ]:
tvec = np.arange(0,10,0.1)
s_low = scipy.integrate.odeint(
    gradfun,
    y0 = 1.5,
    t = tvec,
    args = (params,))
s_hi = scipy.integrate.odeint(
    gradfun,
    np.array(2.2),
    t = tvec,
    args = (params,))

In [ ]:
%matplotlib inline
plt.plot(tvec,s_low, label="N0=1.5")
plt.plot(tvec,s_hi,label="N0=2.2",c="red")
plt.legend();